## Loading libraries

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import librosa, librosa.display, IPython.display as ipd
import numpy as np
import json
from mutagen.mp3 import MP3
from statistics import mean, median
import noisereduce as nr

In [5]:
savePath="./data/xeno-canto-dataset-full-all-Countries/"
bPath = "./data/"
basePath = "data/xeno-canto-dataset"

def save_json(searchTerms, birdName, country):
    numPages = 1
    page = 1
    # create a path to save recordings
    path = bPath + birdName.replace(':', '') + "/" + country
    if not os.path.exists(path):
        print("Creating subdirectory " + path + " for downloaded files...")
        os.makedirs(path)
        # download a json file for every page found in a query
    while page < numPages + 1:
        print("Loading page " + str(page) + "...")
        url = 'https://www.xeno-canto.org/api/2/recordings?query={0}&page={1}'.format(searchTerms.replace(' ', '%20'),
                                                                                      page)
        print(url)
        jsonPage = urllib.request.urlopen(url)
        jsondata = json.loads(jsonPage.read().decode('utf-8'))
        filename = path + "/jsondata_p" + str(page) + ".json"
        with open(filename, 'w') as outfile:
            json.dump(jsondata, outfile)
        # check number of pages
        numPages = jsondata['numPages']
        page = page + 1
    print("Found ", numPages, " pages in total.")
    print("Saved json for ", (numPages - 1) * 500 + len(jsondata['recordings']), " files")
    return path

def read_data(searchTerm, path):
    data = []
    numPages = 1
    page = 1
    # read all pages and save results in a list
    while page < numPages + 1:
        # read file
        with open(path + "/jsondata_p" + str(page) + ".json", 'r') as jsonfile:
            jsondata = jsonfile.read()
        jsondata = json.loads(jsondata)
        # check number of pages
        numPages = jsondata['numPages']
        # find "recordings" in a json and save a list with a search term
        for k in range(len(jsondata['recordings'])):
            data.append(jsondata["recordings"][k][searchTerm])
        page = page + 1
    return data

def download(searchTerms, birdName, country):
    # create data/xeno-canto-dataset directory
    path = save_json(searchTerms, birdName, country)
    # get filenames: recording ID and bird name in latin from json
    filenamesID = read_data('id', path)
    filenamesCountry = read_data('cnt', path)

    fileaddress = read_data('file', path) #json file
    
    numfiles = len(filenamesID)
    print("A total of ", numfiles, " files will be downloaded")
    for i in range(0, numfiles):
        print("Saving file ", i + 1, "/", numfiles,
              path + "/" + birdName.replace(':', '') + filenamesID[i] + ".mp3")
        urllib.request.urlretrieve(fileaddress[i],
                                   path + "/" + birdName + filenamesID[i] + ".mp3")


## Loading data 

In [8]:
countries = ['France']

bird = {
        'gen':'Parus',
        'spec':'major',
        'country':countries
        }

path = './data/rawData/' +bird['gen']+bird['spec']+'/'+bird['country'][0]
idData = read_data('id', path)

## Configuration of mel spectograms

In [9]:
N_FFT = 1024         # Number of frequency bins for Fast Fourier Transform
HOP_SIZE = 1024      # Number of audio frames between STFT columns
SR = 44100           # Sampling frequency
N_MELS = 40          # Mel band parameters   
WIN_SIZE = 1024      # number of samples in each STFT window
WINDOW_TYPE = 'hann' # the windowin function
FEATURE = 'mel'      # feature representation
plt.rcParams['figure.figsize'] = (10,4)

## Creating 5 sec spectograms with reduced noise

In [37]:
import os
import glob

# Define the paths
raw_data_path = './data/rawdata/'
specs_path = './data/specs/'

# Create the specs directory if it doesn't exist
os.makedirs(specs_path, exist_ok=True)

# Define parameters
N_FFT = 1024         # Number of frequency bins for Fast Fourier Transform
HOP_SIZE = 1024      # Number of audio frames between STFT columns
SR = 44100           # Sampling frequency
N_MELS = 40          # Mel band parameters   
WIN_SIZE = 1024      # number of samples in each STFT window
WINDOW_TYPE = 'hann' # the window function

# Get all audio files in the raw data directory
audio_files = glob.glob(raw_data_path + '/**/*.mp3', recursive=True)

for audio_file in audio_files:
    try:
        # Get bird species and country
        species = os.path.basename(os.path.dirname(audio_file))
        country = os.path.basename(os.path.dirname(os.path.dirname(audio_file)))

        # Create directory structure for the spectrogram
        spec_dir = os.path.join(specs_path, species, country, 'spectrogram')
        os.makedirs(spec_dir, exist_ok=True)

        # Load audio
        noisy, sr = librosa.load(audio_file, duration=5, mono=True)

        # Perform noise reduction
        noisy = nr.reduce_noise(y=noisy, y_noise=noisy, prop_decrease = 1, sr=sr)

        # Generate spectrogram
        spec = librosa.feature.melspectrogram(y=noisy, 
                                               sr=sr, 
                                               n_fft=N_FFT, 
                                               hop_length=HOP_SIZE, 
                                               n_mels=N_MELS, 
                                               htk=True, 
                                               fmin=0.0, 
                                               fmax=sr/2.0)

        # Convert to dB scale
        spec_db = librosa.amplitude_to_db(spec, ref=1.0)

        # Plot and save the spectrogram
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(spec_db, sr=sr, hop_length=HOP_SIZE, x_axis='time', y_axis='mel')
        plt.title('Mel spectrogram - ' + os.path.basename(audio_file))
        plt.colorbar(format='%+2.0f dB')
        plt.tight_layout()

        # Save the spectrogram
        spec_filename = os.path.join(spec_dir, os.path.splitext(os.path.basename(audio_file))[0] + '_spec.png')
        plt.savefig(spec_filename)
        plt.close()
    except Exception as e:
        print(f"Error processing {audio_file}: {e}")


Error processing ./data/rawdata\Phoenicurusphoenicurus\France\Phoenicurusphoenicurus737600.mp3: Audio buffer is not finite everywhere
